<a href="https://colab.research.google.com/github/jinkyukim-me/Pytorch_Class_/blob/master/classification_catsdogs_gyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchsummary import summary 

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [0]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
trans = transforms.Compose([transforms.ToTensor()])

train_data = dsets.ImageFolder(root='/content/gdrive/My Drive/project/classification/catsdogs/data/train_data',
                               transform=trans)

In [0]:
#Dataloader
data_loader = DataLoader(dataset = train_data, batch_size = batch_size,
                         shuffle = True, num_workers=2, drop_last=True)

In [0]:
#model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(256, 120),
            nn.ReLU(),
            nn.Linear(120, 2),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        
        return out


In [0]:
model = CNN().to(device)

In [0]:
#loss function
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
#optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
#training
total_batch = len(data_loader)
print('Learning Started. It will takes some time.')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for num, data in enumerate(data_loader): 
        imgs, labels = data
        imgs = imgs.to(device) #input data
        labels = labels.to(device) #label data
        
        optimizer.zero_grad()
        hypothesis = model(imgs) #model에 input값을 넣으면 가설이 됨
        
        cost = criterion(hypothesis, labels) #가설과 실제 라벨 사이의 차이를 계산하는 loss
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch #cost값을 쌓아줌
        
    print('[Epoch:{}] cost = {}'.format(epoch +1, avg_cost))
print('Learning Finished!')

Learning Started. It will takes some time.
[Epoch:1] cost = 0.6936174631118774
[Epoch:2] cost = 0.6935796141624451
[Epoch:3] cost = 0.6935096979141235
[Epoch:4] cost = 0.6934618949890137
[Epoch:5] cost = 0.6933600902557373
[Epoch:6] cost = 0.6933600902557373
[Epoch:7] cost = 0.6933320760726929
[Epoch:8] cost = 0.6933364272117615
[Epoch:9] cost = 0.693314790725708
[Epoch:10] cost = 0.6933146715164185
[Epoch:11] cost = 0.693288266658783
[Epoch:12] cost = 0.6932534575462341
[Epoch:13] cost = 0.6932421922683716
[Epoch:14] cost = 0.6932539343833923
[Epoch:15] cost = 0.6932485699653625
Learning Finished!


In [0]:
trans = transforms.Compose([transforms.ToTensor()])

test_data = dsets.ImageFolder(root='/content/gdrive/My Drive/project/classification/catsdogs/data/test_data',
                               transform=trans)

In [0]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [0]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = model(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())


In [0]:
summary(model, input_size = (3,3,28,28))